# TP 4 Réseaux récurrents 

Réalisé par: 
 - Yacine ALLAOUACHE 
 - Kenza ELMAM 

Notable TODOs:
- Implemenetation d'un RNN 
- Classification de séquences 
- Modéle de séquences multi-variées
- Génération de séquences


# Setup

## Library import
We import all the required Python libraries

In [1]:
# Data manipulation
import pandas as pd
import numpy as np

# Options for pandas
pd.options.display.max_columns = 50
pd.options.display.max_rows = 30

# Visualizations
import plotly
import plotly.graph_objs as go
import plotly.offline as ply
plotly.offline.init_notebook_mode(connected=True)

import cufflinks as cf
cf.go_offline(connected=True)
cf.set_config_file(theme='white')

import matplotlib as plt

# Autoreload extension
if 'autoreload' not in get_ipython().extension_manager.loaded:
    %load_ext autoreload
    
%autoreload 2

## Local library import
We import all the required local libraries libraries

In [2]:
# Include local library paths
import sys
sys.path.append('./src') # uncomment and fill to import local libraries

from  utils import * 
import exo2 
import exo3
import exo4

# Import local libraries
%load_ext autoreload
    
%autoreload 2 

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload



# Data import
We retrieve all the required data for the analysis.

In [3]:
with open(f'{"./data/trump_full_speech.txt"}') as f:
    trump_full_speech = f.read()

In [4]:
temperatures_train = pd.read_csv('./data/tempAMAL_train.csv', low_memory=False)
temperatures_test  = pd.read_csv('./data/tempAMAL_test.csv', low_memory=False, header=None)
temperatures_train.head(5)

,datetime,Vancouver,Portland,San Francisco,Seattle,Los Angeles,San Diego,Las Vegas,Phoenix,Albuquerque,Denver,San Antonio,Dallas,Houston,Kansas City,Minneapolis,Saint Louis,Chicago,Nashville,Indianapolis,Atlanta,Detroit,Jacksonville,Charlotte,Miami,Pittsburgh,Toronto,Philadelphia,New York,Montreal,Boston
0,2012-10-01 13:00:00,284.63,282.08,289.48,281.8,291.87,291.53,293.41,296.6,285.12,284.61,289.29,289.74,288.27,289.98,286.87,286.18,284.01,287.41,283.85,294.03,284.03,298.17,288.65,299.72,281.0,286.26,285.63,288.22,285.83,287.17
1,2012-10-01 14:00:00,284.62904131,282.083251974,289.474992813,281.797216632,291.86818552200003,291.533500952,293.403141271,296.60850854299997,285.154558187,284.607305531,289.303648787,289.762974207,288.297575758,289.997635363,286.893635588,286.18524602900004,284.05469097400004,287.42136028,283.889393939,294.03534141,284.06978923400004,298.205229759,288.650172214,299.732517698,281.024767377,286.262540958,285.663207797,288.24767617,285.834649953,287.186092094
2,2012-10-01 15:00:00,284.626997923,282.091866475,289.460618112,281.789832606,291.86284445900003,291.54335507900004,293.392177052,296.631487354,285.233951595,284.5999178,289.338496754,289.830766948,288.334343434,290.038150556,286.951400772,286.199194111,284.177412183,287.454636935,283.941919192,294.049702185,284.173964682,298.299595186,288.650581705,299.76657946,281.08831873599996,286.269518418,285.756824139,288.326939663,285.847789539,287.23167159
3,2012-10-01 16:00:00,284.624954535,282.100480976,289.446243412,281.78244858,291.857503395,291.55320920599996,293.381212832,296.65446616400004,285.31334500400004,284.59253007,289.373344722,289.89855969,288.371111111,290.07866575,287.009165955,286.213142193,284.300133393,287.48791359,283.994444444,294.064062959,284.278140131,298.393960613,288.650991196,299.800641223,281.151870096,286.27649587900004,285.85044048,288.406203155,285.860929124,287.277251086
4,2012-10-01 17:00:00,284.622911147,282.109095477,289.431868711,281.775064553,291.85216233200003,291.563063333,293.370248613,296.677444975,285.392738413,284.585142339,289.40819268900003,289.96635243099996,288.407878788,290.119180943,287.066931139,286.227090275,284.42285460200003,287.521190245,284.046969697,294.078423734,284.38231558,298.488326039,288.65140068700003,299.83470298599997,281.215421455,286.283473339,285.944056822,288.485466648,285.874068709,287.322830583


# Data processing


Dans cette partie, on supprime les lignes vides dans train et test. 

In [5]:
train = temperatures_train.iloc[:11115, 1:].dropna()
train = pd.concat([train, temperatures_train.iloc[11116:-1, 1:].dropna()], axis=0)
test = temperatures_test.iloc[:, 1:].dropna()


INFO:numexpr.utils:NumExpr defaulting to 8 threads.


# 1. Classification des séquences 

Pour cette tache, on utilisera l'artichitecture many to one, On itere sur la longuer des séquences et on calcule un etat caché qui nous permettra de classer les sequences. 

<img src="img/many_to_one.png" style="display:block;margin-left:auto;margin-right:auto;width:50%;"> <br>

Ensuite, on normalise les données en utilisant StandardScaler

In [6]:
X_train = StandardScaler().fit_transform(train).astype(np.float32)
X_test = StandardScaler().fit_transform(test).astype(np.float32)

# Définition des hyper-parametres 


In [23]:
%%sh
rm -r checkpoints/exo2/
rm -r experiments/exo2/

rm: cannot remove 'checkpoints/exo2/': No such file or directory


In [ ]:
N_LABELS = 10
latent_size = 200
lr = 1e-3
batch_size=16
fixed_length_train = 10
fixed_length_test = 10
n_epochs = 100
log_dir = "./experiments/exo2"
checkpoint_dir = './checkpoints/exo2'
checkpoint_path = f'{checkpoint_dir}/checkpoint_same_l_' + \
            exo2.datetime.now().strftime('%d_%m_%Y_%H:%M:%S')

exo2.os.makedirs(f"{log_dir}", exist_ok=True)
exo2.os.makedirs(f"{checkpoint_dir}", exist_ok=True)

Dans cette partie, on separe les sequences (temperatures) et les labels (les villes)  afin de creer un dataset 

In [25]:
sequences_train_same, labels_train_same = exo2.training_examples(X_train, N_LABELS, length=fixed_length_train)
sequences_test, labels_test =  exo2.training_examples(X_test, N_LABELS, length=fixed_length_test)

In [26]:
data_train = SequencesDatasetWithSameLength(sequences_train_same,labels_train_same)
data_test = SequencesDatasetWithSameLength(sequences_test,labels_test)


In [27]:
trainloader_same = torch.utils.data.DataLoader(
        data_train, batch_size=batch_size, shuffle=True)
testloader_same = torch.utils.data.DataLoader(
        data_test, batch_size=batch_size, shuffle=True)

Création du modele, loss et optimizer

In [28]:
model =  exo2.Rnn_classifier(1,latent_size, N_LABELS).to(device)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

Dans cette partie, on entraine le modele sur nos données, on sauvegarde nos resultats accuracy et loss avec le tensorboard et checkpoints 

In [29]:
losses, accuracies =  exo2.train(
        model=model,
        criterion=criterion,
        optimizer=optimizer,
        dataloader=trainloader_same,
        test_loader=testloader_same,
        n_epochs=n_epochs,
        n_label=N_LABELS,
        device=device,
        log_dir=log_dir,
        checkpoint_path=checkpoint_path
    )

Train: Loss: 2.2361 Acc: 21.1%	Test: Loss: 2.2834 Acc: 14.76%: 100%|██████████| 100/100 [02:04<00:00,  1.24s/it]


On visualise nos resultats, on obtient une accuracy à 13.75% en train et 12% en test. On remarque que notre modele ne se converge pas malgré une exploration des hyper-parametres 

In [91]:
%load_ext tensorboard
%tensorboard --logdir ./experiments/exo2

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 36076), started 3:08:50 ago. (Use '!kill 36076' to kill it.)

# 2. Modéles de séquences multi-variées 

#### Pour cette tache, on utilisera l'artichitecture many to many, On itere sur la longuer de la séquence $T$ et on calcule un etat caché $h_{T}$,  ensuite on applique un decodage à l'instant $T+1$ et/ou $T+2$ pour predire la temperature à $T+1$ et/ou à $T+2$. 

<img src="img/many-to-many.png" style="display:block;margin-left:auto;margin-right:auto;width:50%;"> <br>

# Normalisation des données 

On normalise nos données en utulisant MinMax afin que les valeurs soient entre 0 et 1 ce qui nous permets d'utiliser une sigmoid a la fin sortie de norte modele.

In [31]:
X_train = exo3.MinMaxScaler().fit_transform(train).astype(np.float32)
X_test = exo3.MinMaxScaler().fit_transform(test).astype(np.float32)

## 2.1 RNN commun à toutes les villes 

### Définition des hyper-parametres 


In [194]:
N_LABELS = 10
latent_size = 20
p_temps=2
lr = 1e-4
batch_size=16
fixed_length_train = 27
fixed_length_test = 27
n_epochs = 100
log_dir = "./experiments/exo3/one_rnn"
checkpoint_dir = './checkpoints/exo3/one_rnn'
checkpoint_path = f'{checkpoint_dir}/checkpoint_same_l_' + \
            exo3.datetime.now().strftime('%d_%m_%Y_%H:%M:%S')

exo3.os.makedirs(f"{log_dir}", exist_ok=True)
exo3.os.makedirs(f"{checkpoint_dir}", exist_ok=True)

Dans cette partie, on separe les sequences (temperatures) et les labels (les villes)  afin de creer un dataset 

In [195]:
sequences_train_same, labels_train_same =  exo3.training_examples(X_train, N_LABELS, length=fixed_length_train,pas_de_temps=p_temps)
sequences_test, labels_test = exo3.training_examples(X_test, N_LABELS, length=fixed_length_test,pas_de_temps=p_temps)

In [196]:
data_train = SequencesDatasetWithSameLength(sequences_train_same,labels_train_same)
data_test = SequencesDatasetWithSameLength(sequences_test,labels_test)


In [197]:
trainloader_same = torch.utils.data.DataLoader(
        data_train, batch_size=batch_size, shuffle=True)
testloader_same = torch.utils.data.DataLoader(
        data_test, batch_size=batch_size, shuffle=False)

Création du modele, loss et optimizer

In [198]:
model = exo3.Rnn_forecasting(    
    input_size=1, 
    latent_size=latent_size, 
    output=N_LABELS, 
    pas_de_temp=p_temps
    ).to(device)

criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

Dans cette partie, on entraine le modele sur nos données, on sauvegarde nos resultats accuracy et loss avec le tensorboard et checkpoints 

In [199]:
losses = exo3.train(
        model=model, 
        criterion=criterion, 
        optimizer=optimizer, 
        dataloader=trainloader_same, 
        test_loader=testloader_same, 
        n_epochs=n_epochs,
        log_dir=log_dir,
        checkpoint_path=checkpoint_path)

Train: Loss: 0.0232	Test: Loss: 0.0289: 100%|██████████| 100/100 [08:07<00:00,  4.88s/it]


## 2.1 NN par série de température propre à une ville

### Définition des hyper-parametres 


In [193]:
%%sh
rm -r ./experiments/exo3/
rm -r ./checkpoints/exo3/

In [200]:
n_villes = 10
latent_size = 20
p_temps=2
lr = 1e-4
batch_size=16
fixed_length_train = 27
fixed_length_test = 27
n_epochs = 100
log_dir = "./experiments/exo3/multi_rnn"
checkpoint_dir = './checkpoints/exo3/multi_rnn'
checkpoint_path = f'{checkpoint_dir}/checkpoint_same_l_' + \
            exo3.datetime.now().strftime('%d_%m_%Y_%H:%M:%S')

exo3.os.makedirs(f"{log_dir}", exist_ok=True)
exo3.os.makedirs(f"{checkpoint_dir}", exist_ok=True)

Dans cette partie, on separe les sequences (temperatures) et les labels (les villes)  afin de creer un dataset 

In [201]:
sequences_train_same, labels_train_same =  exo3.training_examples(X_train, N_LABELS, length=fixed_length_train,pas_de_temps=p_temps)
sequences_test, labels_test = exo3.training_examples(X_test, N_LABELS, length=fixed_length_test,pas_de_temps=p_temps)

In [202]:
data_train = SequencesDatasetWithSameLength(sequences_train_same,labels_train_same)
data_test = SequencesDatasetWithSameLength(sequences_test,labels_test)


In [203]:
trainloader_same = torch.utils.data.DataLoader(
        data_train, batch_size=batch_size, shuffle=True)
testloader_same = torch.utils.data.DataLoader(
        data_test, batch_size=batch_size, shuffle=False)

Création du modele, loss et optimizer

In [204]:
model = exo3.MultipleRNN(     
    n_villes = n_villes,
    input_size=1, 
    latent_size=latent_size, 
    pas_de_temp=p_temps
    ).to(device)

criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

Dans cette partie, on entraine le modele sur nos données, on sauvegarde nos resultats accuracy et loss avec le tensorboard et checkpoints 

In [205]:
losses = exo3.trainMR(
        model=model, 
        criterion=criterion, 
        optimizer=optimizer, 
        dataloader=trainloader_same, 
        test_loader=testloader_same, 
        n_epochs=n_epochs,
        log_dir=log_dir,
        checkpoint_path=checkpoint_path)

On visualise nos resultats, on obtient une loss à 0.002% en test.

In [206]:
%load_ext tensorboard
%tensorboard --logdir ./experiments/exo3

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


# 3 Génération de séquences

L’objectif est de décoder à partir de l’état caché une distribution de probabilités multinomiale sur les symboles à engendrer

<img src="img/one_to_many.png" style="display:block;margin-left:auto;margin-right:auto;width:50%;"> <br>

### Définition des hyper-parametres 


In [234]:
latent_size = 100
lr = 1e-4
length=50
batch_size=64
n_epochs = 10
log_dir = "./experiments/exo4"
checkpoint_dir = './checkpoints/exo4'
checkpoint_path = f'{checkpoint_dir}/checkpoint_same_l_' + \
            exo4.datetime.now().strftime('%d_%m_%Y_%H:%M:%S')

exo4.os.makedirs(f"{log_dir}", exist_ok=True)
exo4.os.makedirs(f"{checkpoint_dir}", exist_ok=True)

## Mise en forme des données

In [235]:
text = exo4.unidecode.unidecode(trump_full_speech)
text = exo4.unicodedata.normalize('NFD', text)
text = text.lower()
text = text.translate(str.maketrans("","", exo4.re.sub('[\.|,|;]', '', exo4.string.punctuation)))
text = text.strip()

LETTRES = set(text)
N_LETTRES = len(LETTRES)+1

id2lettre = dict(zip(range(1,len(LETTRES)+1),LETTRES))
lettre2id = dict(zip(id2lettre.values(),id2lettre.keys()))

X = exo4.string2code(text, lettre2id).astype(np.float32)

Dans cette partie, on cree un dataset de sequences ${\{x_1,x_2,...,x_n\}}$ du discours de Trump de taille variable, associée à une traget tel que $(x,y) = (x[:-1], x[1:])$

In [236]:
sequences_train_same, labels_train_same =  exo4.training_examples(X, length, N_LETTRES)

train_datset = SequencesDatasetWithSameLength(sequences_train_same,labels_train_same)

trainloader = torch.utils.data.DataLoader(train_datset, batch_size=batch_size, shuffle=True)

### Definition notre model, optimizer, et la loss function

In [237]:
model = exo4.Rnn_Generator(input_size=N_LETTRES, latent_size=latent_size, output=N_LETTRES).to(device)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=lr)

### Fitting 

In [238]:
exo4.train(
        train=trainloader, 
        model=model, 
        criterion=criterion, 
        optimizer=optimizer, 
        n_lettres=N_LETTRES, 
        n_epochs=n_epochs, 
        log_dir=log_dir, 
        checkpoint_path=checkpoint_path
    )

Train: Loss: 3.6225: 100%|██████████| 10/10 [02:24<00:00, 14.40s/it]


[3.662418092015278,
 3.626484566424266,
 3.6250544657190162,
 3.6246031893305033,
 3.6242653387138644,
 3.6235087567065136,
 3.623019082287708,
 3.622653119535331,
 3.6225445735885438,
 3.62250502425504]

In [239]:
%load_ext tensorboard
%tensorboard --logdir ./experiments/exo4

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6007 (pid 51807), started 0:04:51 ago. (Use '!kill 51807' to kill it.)

In [ ]:
exo4.generate_sequence(
    model=model, 
    start="h", 
    n_lettres=N_LETTRES, 
    length=50, 
    temperature=0.1, 
    lettre2id=lettre2id, 
    id2lettre=id2lettre
    )

In [33]:
import sys
sys.path.append('./implimentation-pl/')
from pytorch_lightning.callbacks import ModelCheckpoint, LearningRateMonitor

from utils import LitModel, RNN, Temperature, nn, pl, torch

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [76]:
class Rnn_classifier(RNN):
    def __init__(self, input_size, latent_size, output):
        super(Rnn_classifier, self).__init__(input_size, latent_size)
        self.out = nn.Linear(latent_size, output)
        self.decision = nn.Softmax(dim=1)

    def decode(self, h):
        x = self.out(h)
        return self.decision(x)
    
class Rnn_forecasting(RNN):
    def __init__(self, input_size, latent_size, pas_de_temp):
        super(Rnn_forecasting, self).__init__(input_size, latent_size)
        self.out = nn.Linear(latent_size, 1)
        self.decision = nn.Sigmoid()
        self.pas_de_temp = pas_de_temp

    def decode(self, last_h):
        result = []
        for _ in range(self.pas_de_temp):
            x = self.out(last_h)
            x = self.decision(x).squeeze(1)
            result.append(x)

        return torch.stack(result, dim=-1)
    
class MultipleRNN(RNN):
    def __init__(self, n_villes, input_size, latent_size, pas_de_temp):
        super(MultipleRNN, self).__init__(input_size, latent_size)
        self.m = torch.nn.ModuleList([Rnn_forecasting(
            input_size=input_size, latent_size=latent_size, pas_de_temp=pas_de_temp) for _ in range(n_villes)])

    def forward(self, X, h=None):
        result = [list(range(len(self.m)))]
        l = []
        for i, model in enumerate(self.m):
            x = X[:, :, i].unsqueeze(-1)            
            out = model(x, model.initHidden(x.shape[0]))
            
            #pred = model.decode(out[-1]).permute(1, 0, 2)
            l.append(out[-1])
        return l#torch.cat(l, dim=-1)
    
    def decode(self, last_h):
        result = []
        for _ in range(self.pas_de_temp):
            x = self.out(last_h)
            x = self.decision(x).squeeze(1)
            result.append(x)

        return torch.stack(result, dim=-1)

In [77]:
backbone = MultipleRNN(5, 1, 30, 1)

In [52]:
datamodule = Temperature(
                seq_length=15,
                batch_size=32,
                data_dir='./data/',
                columns=list(range(5)),
                task=1,
                normalize=True
            )
litmodel=LitModel(backbone, learning_rate=1e-3, forcasting=True)

In [44]:
trainer = pl.Trainer(progress_bar_refresh_rate=25)

GPU available: False, used: False
TPU available: None, using: 0 TPU cores


In [45]:
trainer.fit(litmodel, datamodule=datamodule)


  | Name      | Type             | Params
-----------------------------------------------
0 | backbone  | Rnn_forecasting  | 1.0 K 
1 | criterion | MSELoss          | 0     
2 | metric    | MeanSquaredError | 0     
-----------------------------------------------
1.0 K     Trainable params
0         Non-trainable params
1.0 K     Total params
0.004     Total estimated model params size (MB)
/home/melissa/anaconda3/envs/main/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:49: UserWarning: The dataloader, val dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


/home/melissa/anaconda3/envs/main/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:49: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


/home/melissa/anaconda3/envs/main/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:49: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  warnings.warn(*args, **kwargs)


1

In [53]:
datamodule.setup()

In [78]:
for x, y in datamodule.train_dataloader():
    out = backbone(x, backbone.initHidden(x.shape[0]))
    #logits = backbone.decode(out[-1])
    break

IndexError: index 1 is out of bounds for dimension 2 with size 1

In [8]:
from pytorch_lightning.metrics import MeanSquaredError

In [12]:
MeanSquaredError()(torch.randn(32,2),torch.randn(32,2))

tensor(1.6892)